In [4]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd

In [5]:
def calculate_mean_std(loader):
    # Variances and means across channels
    mean = torch.zeros(3)
    std = torch.zeros(3)
    nb_samples = 0
    
    # Calculate mean
    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        nb_samples += batch_samples
    
    mean /= nb_samples
    
    # Calculate std
    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        std += ((images - mean.unsqueeze(1))**2).mean(2).sum(0)
    
    std = torch.sqrt(std / nb_samples)
    return mean, std


In [6]:
def calcNormalization(batch_size):
    # Define transformations, adjust according to your need
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize all images to 256x256
        transforms.ToTensor(),  # Convert images to PyTorch tensors
    ])

    # Get the path to the current working directory
    base_dir = os.getcwd()
    # Append subdirectory to the base path
    train_path = os.path.join(base_dir, 'ImageData', 'train')

    # Loading the training data
    train_data = datasets.ImageFolder(root=train_path, transform=transform)
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    mean, sd = calculate_mean_std(train_loader)
    return mean, sd

In [7]:
mean, sd = calcNormalization(1)

transform = transforms.Compose([
    transforms.Resize((256, 256)),  # Resize all images to 256x256
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=mean.tolist(), std=sd.tolist())
])

# Get the path to the current working directory
base_dir = os.getcwd()

# Append subdirectory to the base path
train_path = os.path.join(base_dir, 'ImageData', 'train')
test_path = os.path.join(base_dir, 'ImageData', 'test')

# Loading the training data
train_data = datasets.ImageFolder(root=train_path, transform=transform)
train_loader = DataLoader(train_data, batch_size=22, shuffle=True)

# Loading the test data
test_data = datasets.ImageFolder(root=test_path, transform=transform)
test_loader = DataLoader(test_data, batch_size=5, shuffle=False)

In [9]:
print(train_data[0])

(tensor([[[-2.0225, -2.0385, -1.9271,  ..., -1.8475, -1.7520, -1.8316],
         [-2.0544, -1.9907, -1.8952,  ..., -1.8634, -1.7679, -1.7998],
         [-1.4656, -1.2746, -1.1155,  ..., -1.8793, -1.8475, -1.7679],
         ...,
         [-1.7838, -1.6247, -1.6406,  ..., -1.9589, -1.9271, -1.9748],
         [-1.7520, -1.6565, -1.6406,  ..., -1.9907, -1.9907, -1.9430],
         [-1.7520, -1.7838, -1.8316,  ..., -2.0225, -1.9907, -1.9748]],

        [[-2.0314, -2.0474, -1.9351,  ..., -1.8548, -1.7586, -1.8388],
         [-2.0635, -1.9993, -1.9030,  ..., -1.8709, -1.7746, -1.8067],
         [-1.4697, -1.2772, -1.1167,  ..., -1.8869, -1.8548, -1.7746],
         ...,
         [-1.7907, -1.6302, -1.6462,  ..., -1.9672, -1.9351, -1.9832],
         [-1.7586, -1.6623, -1.6462,  ..., -1.9993, -1.9993, -1.9511],
         [-1.7586, -1.7907, -1.8388,  ..., -2.0314, -1.9993, -1.9832]],

        [[-1.9997, -2.0152, -1.9068,  ..., -1.8293, -1.7364, -1.8138],
         [-2.0307, -1.9687, -1.8758,  ..., -

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [6]:
# Load a pre-trained ResNet18 model
model = models.resnet18(weights='ResNet18_Weights.DEFAULT')

# Modify the final fully connected layer to match the number of classes in your dataset
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Move the model to the chosen device
model = model.to(device)

In [7]:
# Loss function
criterion = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 12
# Training loop
for epoch in range(num_epochs):  # num_epochs should be set by you
    model.train()  # Set model to training mode
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')

Epoch 1, Loss: 0.5716857686638832
Epoch 2, Loss: 0.30265215784311295
Epoch 3, Loss: 0.18266824260354042
Epoch 4, Loss: 0.12496978044509888
Epoch 5, Loss: 0.07081027328968048
Epoch 6, Loss: 0.04670743551105261
Epoch 7, Loss: 0.03066325467079878
Epoch 8, Loss: 0.05198593670502305
Epoch 9, Loss: 0.05561822606250644
Epoch 10, Loss: 0.016229575034230947
Epoch 11, Loss: 0.018644500290974975
Epoch 12, Loss: 0.014191812137141824


In [8]:
model.eval()
correct = 0
total = 0
y_pred = []
y_true = []
for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    y_pred.extend(predicted.cpu().numpy())
    y_true.extend(labels.cpu().numpy())

print(classification_report(y_true, y_pred))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

[[5 0]
 [0 5]]


In [9]:
torch.save(model.state_dict(), 'image_model.pth')